## Esse notebook faz a integração final de todos os arquivos que limpei e padronizei. 
### ele vai: 
- Ler todos os arquivos .parquet que foram gerados.
- Faz o merge e juntar tudo em uma unica tabela
- Calcular as taxas (Engenharia de Features) usando a população e o PIB
- Gerar o arquivo final para a Modelagem.

importação de bibliotecas 

In [2]:
import pandas as pd
import numpy as np
import duckdb
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# Caminhos
PROCESSED_PATH = Path('../data/processed')
GOLD_PATH = Path('../data/gold') # Pasta para o arquivo finalzão ("Ouro")
GOLD_PATH.mkdir(exist_ok=True, parents=True)

print("Ambiente pronto.")

Ambiente pronto.


Carregar Bases Processadas

In [ ]:
print("Carregando bases processadas e PADRONIZANDO CHAVES...")

# Função auxiliar para garantir chave padrao
def padronizar_chave(df):
    # Garante que codmun seja texto de 6 digitos e ano seja inteiro
    df['codmun'] = df['codmun'].astype(str).str.split('.').str[0].str.zfill(6).str.slice(0, 6)
    df['ano'] = df['ano'].astype(int)
    return df

# 1. População
df_pop = pd.read_parquet(PROCESSED_PATH / 'populacao_completa.parquet')
df_pop = padronizar_chave(df_pop)
print(f" População: {df_pop.shape}")

# 2. Mortalidade
df_mort = pd.read_parquet(PROCESSED_PATH / 'mortalidade_cardio_clean.parquet')
df_mort = padronizar_chave(df_mort)
print(f" Mortalidade: {df_mort.shape}")

# 3. CNES
df_leitos = pd.read_parquet(PROCESSED_PATH / 'cnes_leitos_clean.parquet')
df_leitos = padronizar_chave(df_leitos)

df_medicos = pd.read_parquet(PROCESSED_PATH / 'cnes_medicos_clean.parquet')
df_medicos = padronizar_chave(df_medicos)

df_estab = pd.read_parquet(PROCESSED_PATH / 'cnes_estabelecimentos_clean.parquet')
df_estab = padronizar_chave(df_estab)
print(f" CNES Carregado")

# 4. IDHM
df_idhm = pd.read_parquet(PROCESSED_PATH / 'idhm_final.parquet')
df_idhm = padronizar_chave(df_idhm)
# Remove colunas duplicadas de nome se houver
cols_idhm_drop = ['nome_origem', 'nome_limpo', 'chave', 'nome_municipio']
df_idhm = df_idhm.drop(columns=[c for c in cols_idhm_drop if c in df_idhm.columns])
print(f" IDHM: {df_idhm.shape}")

# 5. PIB (Onde deu erro)
try:
    df_pib = pd.read_parquet(PROCESSED_PATH / 'pib_total.parquet')
    df_pib = padronizar_chave(df_pib)
    
    # Garantir que o VALOR do PIB seja numérico
    # O IBGE as vezes manda '-' ou '...' como texto
    df_pib['pib_total'] = pd.to_numeric(df_pib['pib_total'], errors='coerce')
    
    print(f" PIB: {df_pib.shape} (Amostra valor: {df_pib['pib_total'].iloc[0]})")
except:
    print(" PIB não encontrado.")
    df_pib = None

Carregando bases processadas...
População carregada: (16113, 8)
Mortalidade carregada: (16674, 3)
Leitos carregados: (10915, 4)
Médicos carregados: (16698, 4)
Estabelecimentos carregados: (16699, 4)
IDHM carregado: (16113, 8)
PIB carregado: (16710, 3)


Hora de fazer o merge 

In [4]:
# Padronizar chaves para o merge
chave = ['codmun', 'ano']

print("Iniciando Merge em Cascata...")

# Usamos População como base (Left Join) pois ela tem a malha completa de municípios
df_final = df_pop.merge(df_mort, on=chave, how='left') \
                 .merge(df_leitos[['codmun', 'ano', 'leitos_sus']], on=chave, how='left') \
                 .merge(df_medicos[['codmun', 'ano', 'total_medicos']], on=chave, how='left') \
                 .merge(df_estab[['codmun', 'ano', 'estabelecimentos']], on=chave, how='left') \
                 .merge(df_idhm, on=chave, how='left')

# Se tiver PIB, junta também
if df_pib is not None:
    df_final = df_final.merge(df_pib[['codmun', 'ano', 'pib_total']], on=chave, how='left')

# --- TRATAMENTO DE NULOS PÓS-MERGE ---

# 1. Métricas de Contagem (Mortes, Leitos, Médicos) -> Se não tem registro, é 0
cols_zeros = ['mortes_cardio', 'leitos_sus', 'total_medicos', 'estabelecimentos']
for col in cols_zeros:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0)

# 2. Métricas Socioeconômicas (IDHM, PIB) -> Se faltar, tentamos preencher com vizinho temporal
# Agrupa por cidade e preenche (Forward Fill depois Backward Fill)
cols_fill = ['idhm', 'renda_pc', 'tx_analfabetismo', 'idhm_renda', 'idhm_educ', 'idhm_longevidade']
if 'pib_total' in df_final.columns:
    cols_fill.append('pib_total')

print("Preenchendo dados socioeconômicos faltantes...")
for col in cols_fill:
    if col in df_final.columns:
        # Preenche buracos usando dados da própria cidade em outros anos
        df_final[col] = df_final.groupby('codmun')[col].ffill().bfill()

# Remover cidades que ainda ficaram com nulos críticos (provavelmente erros de código IBGE que não casaram)
df_final = df_final.dropna(subset=['idhm'])

print(f"Dataset Integrado Final: {df_final.shape}")
df_final.head()

Iniciando Merge em Cascata...
Preenchendo dados socioeconômicos faltantes...
Dataset Integrado Final: (16783, 19)


,ano,nome_origem,populacao,tx_envelhecimento,nome_limpo,codmun,nome_temp,nome_norm,mortes_cardio,leitos_sus,total_medicos,estabelecimentos,idhm,idhm_renda,idhm_longevidade,idhm_educ,renda_pc,tx_analfabetismo,pib_total
0,2007,Alta Floresta D'Oeste (RO),26533.0,4.595,None,110001,None,None,21.0,45.0,180.0,12.0,0.483,0.637,0.7305,0.3940,424.07,15.21,NaN
1,2010,Alta Floresta D'Oeste (RO),24392.0,5.840,ALTA FLORESTA D'OESTE,110001,Alta Floresta D'Oeste,ALTA FLORESTA D'OESTE,16.0,45.0,202.0,15.0,0.641,0.657,0.7630,0.5260,476.99,13.00,NaN
2,2015,Alta Floresta D'Oeste (RO),24392.0,5.840,None,110001,None,None,32.0,45.0,263.0,24.0,0.641,0.657,0.7630,0.5260,476.99,13.00,NaN
3,2007,Ariquemes (RO),74503.0,3.640,None,110002,None,None,69.0,66.0,555.0,44.0,0.556,0.695,0.7740,0.4715,610.41,10.67,NaN
4,2010,Ariquemes (RO),90353.0,4.360,ARIQUEMES,110002,Ariquemes,ARIQUEMES,98.0,66.0,825.0,84.0,0.702,0.716,0.8060,0.6000,689.95,8.53,NaN


Engenharia de Features (Criação das Taxas)

In [5]:
print("Criando novas features (Taxas padronizadas)...")

# 1. Target (Mortalidade por 100k habitantes)
df_final['taxa_mortalidade'] = (df_final['mortes_cardio'] / df_final['populacao']) * 100000

# 2. Infraestrutura de Saúde
# Leitos por 1.000 habitantes (Padrão OMS)
df_final['leitos_por_mil'] = (df_final['leitos_sus'] / df_final['populacao']) * 1000

# Médicos por 100k habitantes
df_final['medicos_por_100k'] = (df_final['total_medicos'] / df_final['populacao']) * 100000

# Estabelecimentos por 100k habitantes
df_final['estab_por_100k'] = (df_final['estabelecimentos'] / df_final['populacao']) * 100000

# 3. Indicador Econômico (PIB per Capita calculado)
if 'pib_total' in df_final.columns:
    # O PIB do IBGE (Tabela 5938) geralmente vem em "Mil Reais" (x1000)
    # Fórmula: (PIB * 1000) / População = Reais por pessoa
    df_final['pib_per_capita_calc'] = (df_final['pib_total'] * 1000) / df_final['populacao']

# 4. Variável Binária (Dummy): Tem Hospital?
# Consideramos "Ter Hospital" se tiver pelo menos 10 leitos SUS (para filtrar postinhos pequenos)
df_final['tem_hospital'] = (df_final['leitos_sus'] >= 10).astype(int)

# 5. Extrair UF e Região do Código IBGE
# O primeiro dígito é a Região, os dois primeiros são a UF
# Regiões: 1=Norte, 2=Nordeste, 3=Sudeste, 4=Sul, 5=Centro-Oeste
mapa_regiao = {'1': 'Norte', '2': 'Nordeste', '3': 'Sudeste', '4': 'Sul', '5': 'Centro-Oeste'}

df_final['cod_uf'] = df_final['codmun'].str.slice(0, 2)
df_final['cod_regiao'] = df_final['codmun'].str.slice(0, 1)
df_final['regiao'] = df_final['cod_regiao'].map(mapa_regiao)

print("Features criadas com sucesso!")
# Mostrar estatísticas das novas variáveis para ver se não tem nada absurdo (ex: taxa infinita)
print(df_final[['taxa_mortalidade', 'leitos_por_mil', 'pib_per_capita_calc']].describe())

Criando novas features (Taxas padronizadas)...
Features criadas com sucesso!
       taxa_mortalidade  leitos_por_mil  pib_per_capita_calc
count      16783.000000    16783.000000                  0.0
mean         214.214812        1.822023                  NaN
std          805.423931        5.305001                  NaN
min            0.000000        0.000000                  NaN
25%          124.106493        0.000000                  NaN
50%          175.447147        1.248561                  NaN
75%          230.000720        2.365876                  NaN
max        61599.696740      241.470811                  NaN


Salvar Dataset

In [6]:
# Salvar Parquet (Otimizado para leitura no notebook de Modelagem)
arquivo_ouro_parquet = GOLD_PATH / 'dataset_modelagem.parquet'
df_final.to_parquet(arquivo_ouro_parquet, index=False)

# Salvar CSV (Para conferência visual/Excel)
arquivo_ouro_csv = GOLD_PATH / 'dataset_modelagem.csv'
df_final.to_csv(arquivo_ouro_csv, index=False)

print(f"🏆 Dataset Final Salvo em:\n - {arquivo_ouro_parquet}\n - {arquivo_ouro_csv}")

# --- Validação SQL (Requisito 2.3 / 3.3) ---
print("\n=== Validação Final via SQL (DuckDB) ===")
con = duckdb.connect()
con.register('tb_final', df_final)

# Consulta: Correlação Visual (Validar hipótese: Mais IDHM = Menos Morte?)
query_validacao = """
SELECT 
    ano,
    COUNT(*) as qtd_municipios,
    AVG(taxa_mortalidade)::INT as media_taxa_mortes,
    AVG(pib_per_capita_calc)::INT as media_pib,
    CORR(idhm, taxa_mortalidade)::DECIMAL(5,2) as corr_idhm_morte
FROM tb_final
GROUP BY ano
ORDER BY ano
"""
print(con.execute(query_validacao).df())

🏆 Dataset Final Salvo em:
 - ../data/gold/dataset_modelagem.parquet
 - ../data/gold/dataset_modelagem.csv

=== Validação Final via SQL (DuckDB) ===
    ano  qtd_municipios  media_taxa_mortes  media_pib  corr_idhm_morte
0  2007            5277                189       <NA>             0.07
1  2010            6229                246       <NA>             0.03
2  2015            5277                202       <NA>             0.02
